In [2]:
pip install gensim

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
pip install word2vec

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for word2vec: filename=word2vec-0.11.1-cp37-none-any.whl size=179688 sha256=972dc329fa6f87fd028cb9e4eb9435b75e616ddd136deb0f73b887e8e83fe8ee
  Stored in directory: C:\Users\paltu\AppData\Local\pip\Cache\wheels\f3\7c\ac\fcb6d867f806021c3730fd848970db988b1d0030b5d20c0e02
Successfully built word2vec
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [12]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [16]:
df1=pd.read_csv("phrases.csv", encoding='latin1')
df1.head()

,Phrases
0,how company compares to its peers?
1,what is the detailed income statement breakdow...
2,world premium penetration in 2020
3,How does the forecasted insurance premium pene...
4,what are the total losses for companies in cou...


In [15]:
import gensim
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load the Word2Vec model from your directory i.e. model_path
model = gensim.models.KeyedVectors.load_word2vec_format('model_path/GoogleNews-vectors-negative300.bin', binary=True)

# Load phrases from the CSV file (assumes a 'phrase' column in the CSV)
phrases_df = pd.read_csv("phrases.csv", encoding='latin1')
phrases = phrases_df['Phrases'].tolist()

# Assign Word2Vec embeddings to each word in the phrases
def get_phrase_embedding(phrase, model):
    words = phrase.split()
    valid_words = [word for word in words if word in model]
    if valid_words:
        phrase_embedding = np.mean([model[word] for word in valid_words], axis=0)
        return phrase_embedding
    else:
        return None

# Create a dictionary to store phrase embeddings
phrase_embeddings = {}
for phrase in phrases:
    embedding = get_phrase_embedding(phrase, model)
    if embedding is not None:
        phrase_embeddings[phrase] = embedding

# Batch execution: Calculate cosine similarities between all pairs of phrases
def calculate_similarity_matrix(phrase_embeddings, phrases):
    similarity_matrix = np.zeros((len(phrases), len(phrases)))
    for i in range(len(phrases)):
        for j in range(len(phrases)):
            similarity = cosine_similarity([phrase_embeddings[phrases[i]]], [phrase_embeddings[phrases[j]]])[0][0]
            similarity_matrix[i][j] = similarity
    return similarity_matrix

similarity_matrix = calculate_similarity_matrix(phrase_embeddings, phrases)

# On-the-fly execution: Find the closest match to a user-input phrase
def find_closest_match(user_input, phrase_embeddings, phrases):
    user_embedding = get_phrase_embedding(user_input, model)
    if user_embedding is not None:
        similarities = [cosine_similarity([user_embedding], [phrase_embeddings[phrase]])[0][0] for phrase in phrases]
        closest_match_idx = np.argmax(similarities)
        closest_match = phrases[closest_match_idx]
        distance = 1 - similarities[closest_match_idx]
        return closest_match, distance
    else:
        return "User input has no valid words for embeddings.", None

# Example usage
user_input = "I need help with data analysis"
closest_match, distance = find_closest_match(user_input, phrase_embeddings, phrases)
print(f"Closest Match: {closest_match}, Distance: {distance:.4f}")


Closest Match: how does economic profit for in country compare to others?, Distance: 0.4662


# Examples

In [16]:
user_input = "oil rigs in the world"
closest_match, distance = find_closest_match(user_input, phrase_embeddings, phrases)
print(f"Closest Match: {closest_match}, Distance: {distance:.4f}")

Closest Match: Show me all the oil rigs in the world, Distance: 0.1418


In [17]:
user_input = "how comapny compares to its peer"
closest_match, distance = find_closest_match(user_input, phrase_embeddings, phrases)
print(f"Closest Match: {closest_match}, Distance: {distance:.4f}")

Closest Match: how company compares to its peers?, Distance: 0.1062


In [21]:
user_input = "Brazilian Insurance mArket"
closest_match, distance = find_closest_match(user_input, phrase_embeddings, phrases)
print(f"Closest Match: {closest_match}, Distance: {distance:.4f}")

Closest Match: Insurance premiums market in Country, Distance: 0.4076


**Tkinter GUI**

In [ ]:
import gensim
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import tkinter as tk
from tkinter import ttk

# Load the Word2Vec model
model = gensim.models.KeyedVectors.load_word2vec_format('C:/Users/paltu/anaconda_copy/GoogleNews-vectors-negative300.bin', binary=True)

# Load phrases from the CSV file (assumes a 'phrase' column in the CSV)
phrases_df = pd.read_csv("phrases.csv", encoding='latin1')
phrases = phrases_df['Phrases'].tolist()

# Assign Word2Vec embeddings to each word in the phrases
def get_phrase_embedding(phrase, model):
    words = phrase.split()
    valid_words = [word for word in words if word in model]
    if valid_words:
        phrase_embedding = np.mean([model[word] for word in valid_words], axis=0)
        return phrase_embedding
    else:
        return None

# Create a dictionary to store phrase embeddings
phrase_embeddings = {}
for phrase in phrases:
    embedding = get_phrase_embedding(phrase, model)
    if embedding is not None:
        phrase_embeddings[phrase] = embedding

# On-the-fly execution: Find the closest match to a user-input phrase
def find_closest_match(user_input, phrase_embeddings, phrases):
    user_embedding = get_phrase_embedding(user_input, model)
    if user_embedding is not None:
        similarities = [cosine_similarity([user_embedding], [phrase_embeddings[phrase]])[0][0] for phrase in phrases]
        closest_match_idx = np.argmax(similarities)
        closest_match = phrases[closest_match_idx]
        distance = 1 - similarities[closest_match_idx]
        return closest_match, distance
    else:
        return "User input has no valid words for embeddings.", None

# Tkinter GUI
def find_closest_match_gui():
    user_input = user_input_entry.get()
    closest_match, distance = find_closest_match(user_input, phrase_embeddings, phrases)
    result_label.config(text=f"Closest Match: {closest_match}\nDistance: {distance:.4f}")

app = tk.Tk()
app.title("Word2Vec Phrase Similarity -- Made by Tushar Pal")

user_input_label = ttk.Label(app, text="Enter a phrase:")
user_input_label.pack(pady=10)

user_input_entry = ttk.Entry(app)
user_input_entry.pack()

find_button = ttk.Button(app, text="Find Closest Match", command=find_closest_match_gui)
find_button.pack(pady=10)

result_label = ttk.Label(app, text="")
result_label.pack()

app.mainloop()
